### Quick Description

Substitutes unknown tokens by the special token `__fuckin__`

[Swear Words Dictionary](https://www.noswearing.com/dictionary/)

In [1]:
import os
import glob
import yaml
import time
import json
import sys
import re
import pandas as pd

from tqdm import tqdm
from googleapiclient import discovery

sys.path.append("../utils")
from toxicity_api_communication import get_toxicity_score

In [30]:
filepaths = yaml.load(open("/home/guilherme/Desktop/dissertation/config/filepaths.yaml"))
credentials = yaml.load(open("/home/guilherme/Desktop/dissertation/config/credentials.yaml"))["perspective-api"]

service = discovery.build(
    "commentanalyzer",
    "v1alpha1",
    developerKey=credentials["key-1"],
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
)

/tmp/ipykernel_102897/1225555520.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  filepaths = yaml.load(open("../config/filepaths.yaml"))
/tmp/ipykernel_102897/1225555520.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credentials = yaml.load(open("../config/credentials.yaml"))["perspective-api"]


---
### Replace tokens with curse words

In [32]:
inputted_curse_words = []
filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(filepaths["01_youtube_raw"], "*"))]
for filename in tqdm(filenames, total=len(filenames)):
    with open(os.path.join(filepaths["01_youtube_raw"], filename)) as file:
        text = file.read()
        
    inputted_curse_words.append((filename, 1 if "[ __ ]" in text else 0))

    text = re.sub(r"\[ __ \]", "__fuckin__", text).strip()
    text = re.sub(r"\[(.*?)\]", '', text).strip()
        
    with open(os.path.join(filepaths["02_youtube_preprocessed"], filename), 'w') as file:
        file.write(text)

df_inputation_schema = pd.DataFrame(inputted_curse_words, columns=["filename", "inputted"])

100%|████████████████████████████████████████████████████████████| 634/634 [00:00<00:00, 19880.30it/s]


In [33]:
df_inputation_schema.to_csv(filepaths["00_inputation_schema"], index=False)